In [8]:
using LinearAlgebra
using SparseArrays

In [9]:
n = 6
M = 3
L = 2
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
# RHS
B = randn(n, L);

In [10]:
# Initialize
X_σ     = zeros(n, L) 
X_exact = zeros(n, L)
Res     = zeros(M, L);

In [11]:
iter(i, p) = (i-1)*p+1:i*p
# Block Arnoldi based on 
#   Iterative Methods for Sparse Linear Systems, Yousef Saad
function bArnoldi(A, X0, m, p)
    n = size(A, 1)
    U = zeros(n, p*(m+1))
    H = zeros(p*(m+1), p*m)
    for i=1:p
        U[:, i] = X0[:, i] / norm(X0[:, i])
    end
    for j in 1:m
        iterJ = iter(j, p)
        W = A * U[:, iterJ]
        for i in 1:j
            iterI = iter(i, p)
            H[iterI, iterJ] = U[:, iterI]' * W
            W -= U[:, iterI] * H[iterI, iterJ]
        end
        iterJp1 = iter(j+1, p)
        Q, R  = qr(W)
        U[:, iterJp1] = Q[:, 1:p]
        H[iterJp1, iterJ] = R
    end
    U, H
end

bArnoldi (generic function with 1 method)

In [12]:
bArnoldi(A, B, M, L)

([-0.7493558369292218 -0.0885312695187261 … -0.153728919508471 0.10851458369086342; -0.22182009918222256 0.13958406222142467 … 0.3807677309760626 -0.1809152467477373; … ; 0.3224606929934708 0.7069480844233842 … -0.348886069167206 0.5115670039388627; 0.2575658240294458 -0.6347564087477443 … 0.6086520899994038 0.4463937962003504], [10.939174015783495 2.3203391244088265 … 2.808034259803625 8.414895203403393; -0.1390424123675338 16.759187781407377 … 14.36227323589153 -1.5544989957850832; … ; 0.0 0.0 … 5.659942381411821 0.7144799350002932; 0.0 0.0 … 0.0 0.5677600294127558])